In [1]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [31]:
def invertindex(docs="docs.txt"):
    '''This fundtion will take the document txt folder (each line should indicate new document and the first token is document name)
    It will return the inverted index and the tokenized (and stemed) version of all the documents'''
    stemmed=[]
    infile = open(docs,'r')
    docs=infile.readlines()
    docs=[i.strip() for i in docs] #removing the \n
    docs=[i.lower() for i in docs] #Lower Case
    tokenized = [word_tokenize(docs[i]) for i in range(len(docs))]
    [i.pop(0) for i in tokenized] # Removing the ID
    for i in range(len(docs)):
        stem = [porter.stem(word) for word in tokenized[i]]
        stemmed.append(stem)
    index=dict()
    for i in range(len(stemmed)):
        for word in set(stemmed[i]):
            if word in index:
                index[word].append(i+1)
            else:
                index[word]=[i+1]
    return(index,stemmed)


In [148]:
def positionalintersect(p1,p2,k,invertedindex,tokenized):
    '''This function is for proximity intersection of postings lists p1 and p2. The function finds places where the two terms appear within k words of each other and returns a list of triples giving docID and the term position in p1 and p2.'''
    answer=list()
    shareddoc = set(index[p1]) & set(index[p2]) #Find the shared documents set
    for i in shareddoc :
    #     p1index=tokenized[i-1].index(p1)
    #     p2index=tokenized[i-1].index(p2)   
        p1indices = [i for i, x in enumerate(tokenized[i-1]) if x == p1]
        p2indices = [i for i, x in enumerate(tokenized[i-1]) if x == p2]
        l=list()
        for pp1 in p1indices:
            for pp2 in p2indices:
                while True:
                    if abs(pp1-pp2) <= k:
                        print(abs(pp1-pp2))
                        l.append(pp2)
                    elif pp2 > pp1:
                        break
                    while len(l)!=0 and abs(l[-1] - pp1) > k:
                        l.pop()
                    for s in l:
                        answer.append((i, pp1, s))
                    break
    return(answer)     

In [123]:
def querytokenizer(query):
    '''Given query with following formt: "p1 /k p2"
    This function will return stemed strings of p1 and p2, and k as a intiger'''
    search_token = word_tokenize(query) #Tokenize
    search_token = [i.lower() for i in search_token] # Lower Case
    search_token = [porter.stem(word) for word in search_token] #Stem
    #Extract k
    r = re.compile("\/\d+")
    m = r.findall(query)
    k=int(m[0].replace("/", ""))
    p1=search_token[0]
    p2=search_token[-1]
    return(p1,p2,k)

In [132]:
def submain(docs = "docs.txt"):
    query=input("Please give me the query (format: p1 /k p2)")
    invertedindex,tokenized =invertindex(docs)
    p1,p2,k = querytokenizer(query)
    answer=positionalintersect(p1,p2,k,invertedindex,tokenized)
    print(answer)
    return(answer)

In [135]:
def main():
    while True:
        submain()

In [137]:
main()

Please give me the query (format: p1 /k p2)schizophrenia /4 drug
2
1
[(1, 3, 1), (4, 1, 2)]


KeyboardInterrupt: 

In [4]:
[23,24,25].pop()

25

In [142]:
def positionalintersect2(p1,p2,k,invertedindex,tokenized):
    '''This function is for proximity intersection of postings lists p1 and p2. The function finds places where the two terms appear within k words of each other and returns a list of triples giving docID and the term position in p1 and p2.'''
    answer=list()
    shareddoc = set(index[p1]) & set(index[p2]) #Find the shared documents set
    for i in shareddoc :
    #     p1index=tokenized[i-1].index(p1)
    #     p2index=tokenized[i-1].index(p2)   
        p1indices = [i for i, x in enumerate(tokenized[i-1]) if x == p1]
        p2indices = [i for i, x in enumerate(tokenized[i-1]) if x == p2]
        l=list()
        for pp1 in p1indices:
            for pp2 in p2indices:
                while True:
                    if 0 < pp2-pp1 <= k:
                        print(abs(pp1-pp2))
                        l.append(pp2)
                    elif pp1 > pp2:
                        break
                    elif pp2 > pp1:
                        break
                    while len(l)!=0 and abs(l[-1] - pp1) > k:
                        l.pop()
                    for s in l:
                        answer.append((i, pp1, s))
                    break
    return(answer)     

In [153]:
def submain(docs = "docs.txt"):
    query=input("Please give me the query (format: p1 /k p2)")
    qtype=input("Which type of query do you want? (a for part 1, else for part 2(directional))")
    invertedindex,tokenized =invertindex(docs)
    p1,p2,k = querytokenizer(query)
    if qtype == "a":
        answer=positionalintersect(p1,p2,k,invertedindex,tokenized)
    else:
        answer=positionalintersect2(p1,p2,k,invertedindex,tokenized)
    print(answer)
    return(answer)

In [172]:
submain()

Please give me the query (format: p1 /k p2)breakthrough /2 new
Which type of query do you want? (a for part 1, else for part 2(directional))s
[]


[]

In [159]:
tokenized

[['breakthrough', 'drug', 'for', 'schizophrenia'],
 ['new', 'approach', 'for', 'treatment', 'of', 'schizophrenia'],
 ['new', 'hope', 'for', 'schizophrenia', 'patient'],
 ['new', 'schizophrenia', 'drug']]

In [150]:
index,tokenized=invertindex("docs.txt")
p2='schizophrenia'
p1='drug'
k=2
positionalintersect2(p1,p2,k,index,tokenized)

2


[(1, 1, 3)]

In [129]:
querytokenizer("schizophrenia /221 drug")

('schizophrenia', 'drug', 221)

In [115]:
r = re.compile("\/\d+")
m = r.findall("schizophrenia  /22  drug") #extract k
k=int(m[0].replace("/", ""))
k

22

In [46]:
querytokenizer("schizophrenia /2 drug")

['schizophrenia', '/2', 'drug']

In [41]:
index,tokenized=invertindex("docs.txt")
answer = positionalintersect('schizophrenia','new',5,index,tokenized)
print(answer)

5
3
1
[(2, 5, 0), (3, 3, 0), (4, 1, 0)]


In [24]:
index,tokenized=invertindex("docs.txt")
p1='schizophrenia'
p2='new'
k=3

answer=list()
shareddoc = set(index[p1]) & set(index[p2]) #Find the shared documents set
print (shareddoc)
for i in shareddoc :
#     p1index=tokenized[i-1].index(p1)
#     p2index=tokenized[i-1].index(p2)   
    p1indices = [i for i, x in enumerate(tokenized[i-1]) if x == p1]
    p2indices = [i for i, x in enumerate(tokenized[i-1]) if x == p2]
    l=list()
    for pp1 in p1indices:
        for pp2 in p2indices:
            while True:
                if abs(pp1-pp2) <= k:
                    l.append(pp2)
                    print(l)
                elif pp2 > pp1:
                    print('yes')
                    break
                while len(l)!=0 and abs(l[-1] - pp1) > k:
                    l.pop()
                for s in l:
                    answer.append((i, pp1, s))
                break
                
                

{2, 3, 4}
[0]
[0]


In [23]:
answer

[(3, 3, 0), (4, 1, 0)]

In [42]:
tokenized

[['breakthrough', 'drug', 'for', 'schizophrenia'],
 ['new', 'approach', 'for', 'treatment', 'of', 'schizophrenia'],
 ['new', 'hope', 'for', 'schizophrenia', 'patient'],
 ['new', 'schizophrenia', 'drug']]

In [11]:
index,tokenized=invertindex("docs.txt")
# tokenized[0].append('breakthrough')
tokenized

[['breakthrough', 'drug', 'for', 'schizophrenia'],
 ['new', 'approach', 'for', 'treatment', 'of', 'schizophrenia'],
 ['new', 'hopes', 'for', 'schizophrenia', 'patients'],
 ['new', 'schizophrenia', 'drug']]

In [120]:
tokenized[0].pop()
tokenized[0]

['breakthrough', 'drug', 'for']

In [116]:
indices = [i for i, x in enumerate(tokenized[0]) if x == "breakthrough"]
print(indices)
while i in indices:
    print (i)
    break


In [117]:
for i in indices:
    while True:
        print(i)
        break

0
4


In [35]:
for i in set(index['new']) & set(index['approach']):
    print(i)

2


In [33]:
set(index['new']) & set(index['breakthrough'])

set()

In [26]:
index

{'approach': [2],
 'breakthrough': [1],
 'drug': [1, 4],
 'for': [1, 2, 3],
 'hopes': [3],
 'new': [2, 3, 4],
 'of': [2],
 'patients': [3],
 'schizophrenia': [1, 2, 3, 4],
 'treatment': [2]}

In [29]:
word='breakthrough'
index[word]=[1]

In [39]:
'drug' in index

True

In [43]:
index['drug'].append(2)

In [78]:
index

{'approach': [2],
 'breakthrough': [1],
 'drug': [1, 4],
 'for': [1, 2, 3],
 'hopes': [3],
 'new': [2, 3, 4],
 'of': [2],
 'patients': [3],
 'schizophrenia': [1, 2, 3, 4],
 'treatment': [2]}

In [36]:
a=(i for i in range(len(tokenized)))

In [37]:
a

<generator object <genexpr> at 0x10ef23d58>